<a href="https://colab.research.google.com/github/Aryu-Tamura/GoogleColab_ChatbotTest/blob/main/03_%E8%8B%B1%E4%BC%9A%E8%A9%B1%E3%83%81%E3%83%A3%E3%83%83%E3%83%88%E3%83%9C%E3%83%83%E3%83%88_MVP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**MVP3: 音声での対話機能**

目的: テキスト入力に加えて、音声でAIと対話し、AIの応答を音声で聞けるようにします。

（エラー処理などを省略したシンプルな構成）

手順:

1. 必要なライブラリをインストールします。
2. ライブラリを読み込みます。
3. OpenAI APIキーを設定します。
4. 会話履歴を保存する変数 (`history`) を準備します。
5. 音声認識関数を作成します。
6. 音声合成関数を作成します。
7. メイン関数を作成します。
8. Gradioを使って、シンプルなWebアプリのUIを作ります (`Interface` を使用)。
9. Webアプリを起動します。

--------------------


1. ライブラリのインストール

In [ ]:
!pip install openai
!pip install gradio

------------------

2. ライブラリのインポート



In [ ]:
import gradio as gr
import openai
from google.colab import userdata
import os
import datetime

---------------------------


3. OpenAI APIキーの設定

※事前にColabの左メニュー(🔑)で `OPENAI_API_KEY` という名前でAPIキーを設定しておいてください。



In [ ]:
from google.colab import userdata # APIキーを安全に読み込むため
key = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=key)

----------------------

4. 会話履歴 (`history`) の準備

※このセルはノートブックを開いて最初に一度だけ実行します。

In [ ]:
# グローバル変数として会話履歴を初期化
history = [] # まず空のリストを作成

# AIへの指示（システムプロンプト）を定義
system_prompt = """あなたは、ユーザーと自然で流暢な英会話を行うAI英語教師です。

# あなたの役割
- ユーザーの英語力向上のために、自然で日常的な英会話を行ってください。
- ユーザーの英語表現や文法に間違いがあれば、会話の流れを止めないように、自然な形で指摘し、より適切な表現を提案してください。
- ユーザーの質問や発言には、共感的に、そして前向きな態度で応答してください。

# 回答の形式
- 会話は常に英語で行ってください。日本語で応答してはいけません。
- 回答は、原則として短く（1～3文程度）、自然な口語表現を使ってください。
- ユーザーが文法や表現のミスをした場合は、以下のような形式で訂正例を示してください。
  例 (Example): User: "I goed to park yesterday." AI: "Oh, nice! You mean, 'I went to the park yesterday,' right?"

# 入出力の例 (Input/Output Examples)
User: "Hello! How was your day?" AI: "Hey! My day's going great, thanks for asking. How about you?"
User: "I'm study English today." AI: "That's great! Just a quick tip: you'd say, 'I'm studying English today.' What are you working on right now?"
User: "Can you help me about grammar?" AI: "Of course! I'd say, 'Can you help me with grammar?' Sure thing! What grammar point do you want to talk about?" """

# historyリストの最初にシステムプロンプトを追加
history.append({"role": "system", "content": system_prompt})

print("会話履歴(history)を初期化しました。")

--------------------

5. 音声認識関数



In [ ]:
def speech_to_text_simple(audio_filepath):
    """簡単な音声認識関数 (Whisper)"""
    if audio_filepath is None:
        return ""
    audio_file = open(audio_filepath, "rb")
    transcription = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file
    )
    return transcription.text

--------------------

6. 音声合成関数



In [ ]:
def text_to_speech_simple(text, output_dir="tts_output_simple"):
    """簡単な音声合成関数 (OpenAI TTS)"""
    if not text:
        return None
    try: # 簡単なエラー処理のみ
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        speech_file_path = os.path.join(output_dir, f"response_{timestamp}.mp3")
        response = client.audio.speech.create(
            model="tts-1",
            voice="alloy",
            input=text
        )
        response.stream_to_file(speech_file_path)
        return speech_file_path
    except Exception as e:
        print(f"音声合成エラー: {e}")
        return None

----------------------

7. シンプルなメイン関数

In [ ]:
def english_teacher_bot_simple_voice(input_mode, input_text, input_audio):
    """
    入力モードに応じて処理し、応答テキストと音声パスを返すシンプルな関数。
    戻り値: (実際に使われた入力テキスト, AI応答テキスト, AI応答音声パス)
    """
    global history

    # 1. 入力モードに応じてテキスト化
    processed_input_text = ""
    if input_mode == "Text":
        processed_input_text = input_text
    elif input_mode == "Voice" and input_audio is not None:
        processed_input_text = speech_to_text_simple(input_audio)
    else: # Voiceモードで音声がない場合などは空文字
        processed_input_text = ""

    print(f"処理された入力テキスト: {processed_input_text}")

    # 有効な入力がない場合は終了
    if not processed_input_text:
        return "(入力なし)", "(入力がなかったため応答できません)", None

    # 2. ユーザー入力を履歴に追加
    history.append({"role": "user", "content": processed_input_text})

    # 3. ChatGPT API呼び出し
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=history,
            max_tokens=100,
            temperature=0.7
        )
        assistant_response_text = response.choices[0].message.content
    except Exception as e:
        print(f"ChatGPT APIエラー: {e}")
        assistant_response_text = f"エラー: {e}"
        history.pop() # エラー時はユーザー入力を履歴から削除

    # 4. AI応答を履歴に追加
    history.append({"role": "assistant", "content": assistant_response_text})
    print(f"AI応答テキスト: {assistant_response_text}")

    # 5. 音声合成
    response_audio_path = None
    if not assistant_response_text.startswith("エラー"): # エラーメッセージは読み上げない
        response_audio_path = text_to_speech_simple(assistant_response_text)

    # 6. 結果を返す (入力テキスト, 応答テキスト, 応答音声パス)
    return processed_input_text, assistant_response_text, response_audio_path

print("関数 'english_teacher_bot_simple_voice' を定義しました。")

--------------------

8. Gradio Interface 作成



In [ ]:
# gr.Interface を使ってUIを作成します
bot_interface = gr.Interface(
    fn=english_teacher_bot_simple_voice, # 作成したメイン関数
    # 入力コンポーネントをリストで指定 (順番が重要)
    inputs=[
        gr.Radio(["Text", "Voice"], label="入力方法", value="Text"),
        gr.Textbox(label="テキスト入力", placeholder="テキストを入力するか、下で録音"),
        gr.Audio(sources=["microphone"], type="filepath", label="音声入力")
    ],
    # 出力コンポーネントをリストで指定 (順番が重要)
    outputs=[
        gr.Textbox(label="認識された/入力されたテキスト"), # 戻り値1 (processed_input_text)
        gr.Textbox(label="AIの応答テキスト"),          # 戻り値2 (assistant_response_text)
        gr.Audio(label="AIの応答音声", type="filepath") # 戻り値3 (response_audio_path)
    ],
    title="英会話チャットボット",
    description="テキストまたは音声で入力してください。"
)

print("Gradioインターフェースを作成しました。")

---------------------

9. Webアプリの起動



In [ ]:
# 作成したUIを起動します
bot_interface.launch(share=True)

----------------------

これで、シンプルな構成のMVP3が完成しました。gr.Interface を使用し、音声認識・合成関数やメイン関数も簡略化されています。エラーハンドリングは最小限ですが、基本的な音声入出力の動作を確認できるはずです。